In [ ]:
# 스크롤 하기 전체다 할꺼면 요거로 대체
# 스크롤 전체 내리기
SCROLL_PAUSE_TIME = 1
while True:
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        else:
            last_height = new_height
            continue

In [11]:
# -*- coding: utf-8 -*-

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import openpyxl



# 엑셀 파일 만들기
wb = openpyxl.Workbook()
sheet = wb.active

# 맨 윗줄에 컬럼명 추가
sheet.append(['제목', '카테고리', '메이커', '달성률', '달성액', '서포터수', '좋아요수', '요약글', '목표금액과기간', '글업데이트수', '댓글수', \
              '리워드종류수', '이미지수', '비디오수', '배송시작날짜', '인스타팔로워수', '와디즈팔로워수', '과거프로젝트수', '과거성공프로젝트수'])

# 크롬드라이버 실행 및 로그인
path = "/Applications/chromedriver"
driver = webdriver.Chrome(path)

# 내 계정으로 로그인하기
usr = "nadawj94@naver.com"  # 본인 이메일 쓸 것
pwd = "wj153123!"  # 본인 비번 쓸 것
login_link = 'https://www.wadiz.kr/web/waccount/wAccountLogin?returnUrl=https://www.wadiz.kr/web/main'
driver.get(login_link)
id = driver.find_element_by_css_selector('input#userName')
id.send_keys(usr)
pw = driver.find_element_by_css_selector('input#password')
pw.send_keys(pwd)
login_btn = driver.find_element_by_css_selector('button#btnLogin')
login_btn.click()

time.sleep(1.5)

# 전체 프로젝트 목록 페이지로 가기(종료된 프로젝트들 마감임박순으로 나열한 페이지)
base_link = 'https://www.wadiz.kr/web/wreward/main?keyword=&endYn=Y&order=closing'
driver.get(base_link)
driver.maximize_window()

time.sleep(1)

# 스크롤 하기
no_of_pagedowns = 5  # 페이지 스크롤 횟수
body = driver.find_element_by_tag_name('body')
while no_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.2)
    no_of_pagedowns -= 1

# 개별 프로젝트 컨테이너 찾기
projects = driver.find_elements_by_css_selector('div.ProjectCardList_item__1owJa')

time.sleep(1)


for p in projects:
    idx = 0
    # 전체 목록에서 정보 가져오기
    image = p.find_element_by_css_selector('a.ProjectCardLink_link__2X36I.CommonProjectCard_image__1aEog')
    name = p.find_element_by_css_selector('p.CommonProjectCard_title__28lHZ.RewardProjectCard_title__RDEBu').text
    category = p.find_element_by_css_selector('span.RewardProjectCard_category__1vo_V').text
    maker = p.find_element_by_css_selector('span.RewardProjectCard_makerName__2sITk').text
    percent = p.find_element_by_css_selector('span.RewardProjectCard_percent__edRT9').text.replace('%', '')
    money = p.find_element_by_css_selector('span.RewardProjectCard_amount__2GV5X').text.replace(',', '')
    
    
    
    print(name)


    # 새로운 브라우저 탭에서 프로젝트 세부 페이지 들어가기
    project_url = image.get_attribute('href')

    driver.execute_script("window.open('');")
    time.sleep(1.5)


    # 크롬 드라이버의 포커스를 새로운 탭(세부 페이지)으로 옮기기
    driver.switch_to.window(driver.window_handles[1])
    driver.get(project_url)
    time.sleep(1.5)

    # 서포터 수
    try:
        supporters = driver.find_element_by_css_selector('p.total-supporter strong').text
    except:
        supporters = 0

    # 좋아요(하트) 수
    try:
        likes = driver.find_element_by_css_selector('em.cnt-like').text
    except:
        likes = 0

    # 요약글 텍스트
    try:
        summary = driver.find_element_by_css_selector('div.campaign-summary').text
    except:
        summary = 'None'

    # 프로젝트 목표 금액과 기간
    goal_amount = driver.find_element_by_css_selector('div.wd-ui-campaign-content > div > div:nth-child(4) p').text

    # 새로운 글 업데이트 수
    try:
        new_news = driver.find_element_by_css_selector('ul.tab-list li:nth-of-type(4) span').text
    except:
        new_news = 0

    # 댓글 수(커뮤니티 수)
    try:
        comment_num = driver.find_element_by_css_selector('ul.tab-list li:nth-of-type(5) span').text
    except:
        comment_num = 0

    # 리워드 종류 수
    reward_num = len(driver.find_elements_by_css_selector('button.rightinfo-reward-list'))

    # 소개글 이미지 수
    try:
        img_num = len(driver.find_elements_by_css_selector('div.inner-contents.fr-view img'))
    except:
        img_num = 0

    # 소개글 비디오 수
    try:
        video_num = len(driver.find_elements_by_css_selector('div.ytp-cued-thumbnail-overlay-image'))
    except:
        video_num = 0


    # 펀딩 안내 페이지 들어가기
    funding_info_btn = driver.find_element_by_css_selector('ul.tab-list li:nth-of-type(3) a')
    funding_info_btn.click()

    # 배송 시작 날짜
    delivery_date = driver.find_element_by_css_selector('div#detail-funding-info div.content h3 em').text

    # 커뮤니티 페이지 들어가기
    community_btn = driver.find_element_by_css_selector('ul.tab-list li:nth-of-type(5) a')
    community_btn.click()

    # 댓글(굳이 필요?)

    # 새로운 탭에서 인스타그램 열어 팔로워 수 가져오기
    try:
        instagram = driver.find_element_by_css_selector('ul.social a.instagram')
        instagram_url = instagram.get_attribute('href')

        driver.execute_script("window.open('');")
        time.sleep(1.5)

        driver.switch_to.window(driver.window_handles[2])
        driver.get(instagram_url)
        time.sleep(1.5)
        try:
            # 인스타 팔로워 수
            sns_followers = driver.find_element_by_css_selector('ul.k9GMp  li:nth-of-type(2)  span.g47SY').text
        except:
            # 링크에 에러가 있거나 계정이 삭제된 경우
            sns_followers = 'link error'

        driver.close()
        time.sleep(1.5)

        driver.switch_to.window(driver.window_handles[1])
    except:
        sns_followers = 'no account'

    time.sleep(1.5)


    # 새로운 탭에서 메이커 프로필 페이지 가기
    maker_profile = driver.find_element_by_css_selector('div.maker-info button')
    maker_profile.click()

    time.sleep(1.5)

    # 와디즈 팔로워 수
    wadiz_followers = driver.find_element_by_css_selector('ul.activity-list li:nth-of-type(3) strong').text

    # 과거 리워드 프로젝트 수
    try:
        past_projects_num = len(driver.find_elements_by_css_selector('li.all em.project-type.reward'))-1
    except:
        past_projects_num = 0

    # 과거 성공한 프로젝트 수(현재 포함)
    try:
        past_projects = driver.find_elements_by_css_selector('li.all span.percent')
        n = 0
        for past in past_projects:
            if int(past.text.replace('%', '')) >= 100:
                n += 1
        past_success_projects_num = n
        print(n)
    except:
        past_success_projects_num = 0
        print(past_success_projects_num)


    # 엑셀 시트에 데이터 쓰기
    sheet.append([name, category, maker, percent, money, supporters, likes, summary, goal_amount, new_news, comment_num,\
                  reward_num, img_num, video_num, delivery_date, sns_followers, wadiz_followers, past_projects_num, \
                  past_success_projects_num])


    # 세부 페이지 탭 닫기
    driver.close()
    time.sleep(1.5)

    # 원래 탭(전체 목록 페이지)로 돌아가기
    driver.switch_to.window(driver.window_handles[0])
    
    # 엑셀 파일 저장하기
    fileName = 'Wadiz1{index}{ext}'.format(index=idx,ext='.xlsx')
    wb.save(fileName)
    idx += idx
    

# 크롬드라이버 닫기
driver.close()

print('수집 종료')

# 엑셀 파일 저장하기
wb.save('Wadiz1.xlsx')


최소형 하이파이 오디오 및 낙하산줄 소재의 우퍼 4"스피커 제작
1
위인에 의한, 위인을 위한, 위인;의 (衣) 옷을 만들다.
1
[90개한정] 당신의 인생체어를 소개합니다. 미니라이프 사파리체어
1
[채식 디저트] 콩물로 만든 마카롱이 있다?! 건강간식 피카롱!
2
⭐️꾸덕촉촉⭐️이세상 텐션이 아니다! 브라우니 같은 단백질바, 하루단백바
1
⭐스마트멀티컨트롤러⭐ 키보드+마우스+리모컨 = 루이벨 링크패드
1
[앵콜]옷도 입고, 유기견 아이들에게 행복도 전하는 일석이조 행복한개 티
1
10g, 크고 편안한 순수 티타늄 안경
1
아직도 손에 물 묻히고 청소하세요? 분사와 세탁을 한번에 노터치 밀대걸레
1
[여름철 무더위 완벽대비] 임금님의 선택, 으랏차(茶)!
1
연출하지 않아도 그자체로 멋스러운 오브제같은 '월넛 사이드테이블(협탁)'
1
집들이 선물? 나는 다르다 특별하게 <3D 입체 액자>
0
단짠한 우리인생, 간편하게 즐기는 홈메이드 간장&수제청
1
입체사운드를 넘어 실시간 3D오디오! 4 in 1 무선헤드셋 MRH
1
M자이마와 정수리에 자신감을 채워주는 디앤디(DND) 헤어라인
2
[앵콜] 슬리퍼도 쿠셔닝이다! 우레탄&파일론창을 적용한⭐️쿠션슬리퍼⭐️
7
다양하게 활용하기 좋은 베이직아이템, 아워아더스
0
가죽을 고르고, 실을 고르고, 장식도 고르는 카드지갑
1
[★앵콜★]20초면ok ! 130kcal로 부담없는 쏘핏 곤약이떡볶이!
2
무더운 올 여름 "댕댕이"도 시원하고 상큼하게 [반려동물 쿨매트]
1
8월 초저녁 바람같은 '옐로우 벤치'의 목소리를 들으러 오세요 !
1
두피 온난화 방지 프로젝트! 뿌리는 순간 숲속의 청량함! 후레시에센스
2
MBC 음악감독팀과의 음반 제작, Made in ME 시즌3
4
남자의 아래는 차가워야 하는 법. 당신의 쿨알을 위한 스프레이, 젠츠
3
윤동주시인의 작품을 옷으로 표현하다 [별헤는밤커플한복]
1
[폭풍보습]소주 네 병에도 촉촉한 조대리의 피부비결! 온천수 大시스루패드
1
[올인원 패키지] 검색으로는 알 수 없는 